In [1]:
first_term = "vaderlandsche"

In [2]:
import re, os, glob, pandas as pd
import editdistance
import sys
import dask.dataframe as dd

In [3]:
## Import token totals
os.chdir("C://Users//Ruben//Documents//GitHub//TheForeign//SCC//output-data")
token_totals = pd.read_csv('tokens_year.csv')

## Import Master DF
os.chdir("C://Users//Ruben//Documents//Scriptie//Data//bigrams//output")
df = dd.read_csv('total-bigrams.csv', dtype='str')

##
os.chdir("C://Users//Ruben//Documents//Scriptie//Data//unigrams//output//years")
df_unigrams = pd.read_csv('total-unigrams.csv')

## Define ED function
def GetCloseED(master_df, search_term, len_restr):
    output_list = list()
    
    subset_df = master_df[master_df.ngram.str[0] == search_term[0]]
    
    for word in list(subset_df.ngram):
        ed = editdistance.eval(search_term, word)
        if ed < 3:
            output_list.append(word)
    if len_restr == "yes":
        output_list = [e for e in output_list if len(e) == len(search_term)]
    return output_list

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
lalt = GetCloseED(df, 'vaderlandsche', 'yes')

NotImplementedError: 

In [ ]:
# Extraction based on known position of search word

subset = df[df['ngram'].str.split().str[0].isin(['vaderlandsche'])].compute()
subset = subset.reset_index(drop=True)
subset['count'] = subset['count'].astype(float)
subset = subset.groupby(['ngram','year']).agg('sum')
subset = subset.stack().reset_index().drop('level_2', axis = 1)
subset.columns = ['ngram', 'year', 'count']
subset = subset.dropna()
subset = subset.pivot(index='ngram', columns='year', values='count')
subset = subset.fillna(0)
subset.columns = subset.columns.astype(str)
list_sums = dict()

for i in list(subset.columns):
    tmp = sum(subset[i].astype(float))
    tmp = {i:tmp}
    list_sums.update(tmp)
    
df_sums = pd.DataFrame(list(list_sums.keys()))
df_sums['counts'] = list(list_sums.values())
df_sums.columns = ['years', 'counts']
token_totals['ngram'] = ''

for i in range(0,len(token_totals)):
    year = token_totals.year[i]
    matching_df = df_sums[df_sums.years == str(year)]
    
    if len(matching_df) == 0:
        count = 0
    else:
        count = matching_df.iloc[0,1]
    
    token_totals.ngram[i] = count
token_totals['ngram_rel'] = token_totals['ngram'] / token_totals['tokens'] * 100  

In [12]:
os.chdir("C://Users//Ruben//Documents//GitHub//TheForeign//SCC//output-data")

In [6]:
filename = "bigrams_hetbuitenland" + ".csv"
subset.to_csv(filename, index=True)

In [7]:
token_totals

,year,tokens,ngram,ngram_rel
0,1815,5686482,0,0
1,1816,4802322,0,0
2,1817,4043759,2,4.94589e-05
3,1818,4465745,0,0
4,1819,4242469,3,7.07135e-05
5,1820,4442881,5,0.00011254
6,1821,4561697,0,0
7,1822,4909844,3,6.11017e-05
8,1823,4792725,0,0
9,1824,4911844,0,0
